
# Visium-Clustergrammer2



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
df = {}

from clustergrammer2 import net

import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot
from ipywidgets import HBox

from copy import deepcopy
from glob import glob
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import Voronoi

import warnings
warnings.filterwarnings('ignore')

>> clustergrammer2 backend version 0.5.6


In [3]:
df = pd.read_csv('data/processed/df_meta.csv', index_col=0)
df.shape

(2696, 12)

In [4]:
df['plot-x'] = df['pos-x'].apply(lambda x: x/2 - 700)
df['plot-y'] = df['pos-y'].apply(lambda x: x/2 )

In [5]:
df_gex = pd.read_csv('data/processed/gex.csv', index_col=0)
df_gex.shape

(250, 2696)

In [6]:
x_dim = 2000
y_dim = 2000

fig = plt.figure()

scatter = plt.scatter(df['plot-x'], 
                      df['plot-y'],
                      display_names=False, 
                      default_size=20)


scatter.default_size = 100
scatter.default_opacities = [0.5]*5164

top_margin = 200
inst_width = 800
inst_height = 1000
fig.layout.min_height = str(inst_height) + 'px'
fig.layout.min_width  = str(inst_width) + 'px'

plt.xlim(0, 2.0*x_dim)
plt.ylim(0, 2.0*y_dim)
fig.title = 'Visium Tissue'
fig.fig_margin = {'top': top_margin, 'bottom': 5, 'left': 5, 'right': 5}


In [7]:
# df['gex-umi-sum'].tolist()

In [31]:
def cat_highlight(inst_value):
    
    print(inst_value)
    
    cols = df_gex.columns.tolist()

    if inst_value == 'reset_cats':
        list_opacities = [1 for x in cols]
        # patch.opacities = list_opacities
        scatter.default_opacities = list_opacities
        
    else:
        
        inst_cat = inst_value.split(': ')[1]

        list_opacities = []

        for inst_label in cols:
            inst_opacity = 0.25
            if inst_label == inst_cat:
                inst_opacity = 1
            list_opacities.append(inst_opacity)        

    # patch.opacities = list_opacities
    scatter.default_opacities = [1] * len(cols) # df['gex-umi-sum-norm'].tolist() # list_opacities

In [32]:
def on_value_change(change):
    
    print(change)
    
    if change['new'] == 'null':
        cat_highlight('reset_cats')
        
        fig.title = 'Visium Tissue'
    else: 
        # mousing over category
        if 'cat-' in change['new']:
            inst_cat = change['new'].split(' -> ')[1]
            cat_highlight(inst_cat)       
            
            fig.title = 'Visium Tissue'
            
        # mousing over marker
        else:
            inst_marker = change['new'].split(' -> ')[1]
            ser_opacity = df_gex.loc[inst_marker]
            
            fig.title = 'Visium Tissue: ' + inst_marker
            
            
            # the scatterplot dots are not in the same order as the data in the dataframe
            #list_opacities = [float(x/ser_opacity.max()) for x in list(ser_opacity.get_values())]
            
            # loop up opacities
            list_opacities = []
            rows = ser_opacity.index.tolist()
            for inst_name in df_gex.columns.tolist():
                if inst_name in rows:
                    list_opacities.append( float(ser_opacity[inst_name]/ser_opacity.max())  )
                else:
                    list_opacities.append(0)
                
            scatter.default_opacities = list_opacities


In [33]:
net.viz = net.load_json_to_dict('data/processed/pre-calc-viz.json')
net.widget()
net.widget_instance.observe(on_value_change, names='value')

In [34]:
HBox([fig, net.widget_instance])

reset_cats
reset_cats
